# Algoritmos de Classificação: Regressão Logística

Nós vamos utilizar o dataset Bank Marketing disponibilizado no [site da UCI](http://archive.ics.uci.edu/ml/datasets/Bank+Marketing). Utilizaremos uma versão adaptada para os objetivos da aula e disponível na pasta `data`.

> The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the investment product would be or not subscribed.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                             classification_report, roc_auc_score)

from plotting import (multiple_histograms_plot, plot_confusion_matrix, plot_roc)
from sklearn.externals import joblib

In [3]:
sns.set_context("notebook", font_scale=1.5)

In [4]:
df = pd.read_csv('../data/bank_marketing.csv')

Segue uma descrição sucinta de cada uma das colunas do dataset:

- `duration_seconds`: last contact duration, in seconds (numeric).

- `duration_minutes`: last contact duration, in minutes (numeric).

- `duration_hours`: last contact duration, in hours (numeric).

- `emp.var.rate`: employment variation rate - quarterly indicator (numeric)

- `nr.employed`: number of employees - quarterly indicator (numeric)

- `euribor3m`: euribor 3 month rate - daily indicator (numeric)

- `month`: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

- `contact`: contact communication type (1 for cellular, 2 for telephone) 

- `loan`: has personal loan? (0 for no, 1 for yes)

- `subscribed` - has the client subscribed a term deposit? (True, False)

## Preparando os dados

In [ ]:
# crie a matriz 'X' sem month e o target
<code>

# crie o vetor 'y' com o target
<code>

In [ ]:
# faça um train_test_split com 20% dos dados para teste
# para podermos comparar os resultados entre nós, vamos sempre usar random_state=0

X_train, X_test, y_train, y_test = <code>

## Aplicando a Regressão Logística

In [ ]:
logreg = LogisticRegression()

In [ ]:
# treine o modelo
<code>

# salve as predições do dataset de testes em 'y_pred'
<code>

In [ ]:
y_pred[:5]

In [ ]:
# calcule e imprima a acurácia
<code>

## Avaliação do modelo

### Matriz de Confusão, Precisão, Recall

In [ ]:
# imprima a matriz de confusao com a função 'confusion_matrix'
<code>

In [ ]:
plot_confusion_matrix(y_test, y_pred)

In [ ]:
# imprima precisão, recall e f1-score com a função 'classification_report'
<code>

### Utilizando as probabilidades

In [ ]:
# calcule as probabilidades de investimento com o método 'predict_proba'
# salve em 'y_pred_proba'
<code>

In [ ]:
y_pred_proba = y_pred_proba[:, 1]
y_pred_proba[:5]

In [ ]:
def predictions_hist(y_pred_proba, y_test, density=True):
    preds_df = pd.DataFrame(data=[y_pred_proba, y_test.astype(str)],
                            index=['Prediction', 'True Value']).T

    preds_df['Prediction'] = preds_df['Prediction'].astype(float)
    preds_df['True Value'] = preds_df['True Value'].astype(str)

    multiple_histograms_plot(data=preds_df, x='Prediction', hue='True Value',
                             bins=np.arange(0, 1.1, 0.025), density=density, probability_hist=True)

    return preds_df

In [ ]:
preds_df = predictions_hist(y_pred_proba, y_test)

In [ ]:
preds_df.head()

In [ ]:
# você gostaria de aumentar o recall ou a precisão?
# recalcule a matriz de confusão e o classification report com um threshold customizado
<code>

### Outras métricas populares

In [ ]:
# imprima o AUC com a função 'roc_auc_score'
<code>

In [ ]:
auc_test = plot_roc(y_test, y_pred_proba)

### Identificação de overfitting

In [ ]:
# calcule predições para o dataset de treino
<code>

In [ ]:
# imprima os AUCs de treino e de teste para comparação
<code>

## Tentando melhorar o modelo

Nesta seção, vamos aplicar algumas técnicas que podem resultar em melhores resultados.

### Tratamento de outliers

In [ ]:
# vamos setar a duração máxima de uma call para 3000 segundos
# ou seja, se uma call exceder a duração máxima, edite os valores de acordo

# atenção: é uma boa prática criar uma cópia da estrutura de dados ao invés de modificar a original
# crie uma cópia de 'df' chamada 'df_no_outliers'

<code>

In [ ]:
# faça um novo train-test-split
# chame as variáveis de: X_train_no_outliers, X_test_no_outliers, y_train_no_outliers, y_test_no_outliers
<code>

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train_no_outliers, y_train_no_outliers)
y_pred_proba_no_outliers = logreg.predict_proba(X_test_no_outliers)[:, 1]

In [ ]:
_ = plot_roc(y_test, y_pred_proba_no_outliers)

In [ ]:
_ = predictions_hist(y_pred_proba_no_outliers, y_test_no_outliers)

In [ ]:
def confusion_matrix_report(y_test, y_pred_proba, thres=0.5):
    y_pred_proba_customizado = y_pred_proba >= thres
    print(classification_report(y_test, y_pred_proba_customizado))
    plot_confusion_matrix(y_test_no_outliers, y_pred_proba_customizado)

In [ ]:
# invoque a função 'confusion_matrix_report' com um threshold que 
# otimize a precisão e o recall
<code>

### Lidando com classes desbalanceadas (under-sampling)

In [ ]:
# utilize o método 'sample' do DataFrame Pandas para fazer under-sampling na classe minoritária
# atenção: faça undersampling apenas no treino - o teste tem que ser mantido desbalanceado
<code>

In [ ]:
# confira a taxa de conversão do novo dataset de treino balanceado
<code>

In [ ]:
# crie 'X_train_balanced' e 'y_train_balanced' para uso no modelo
<code>

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train_balanced, y_train_balanced)
y_pred_proba_balanced = logreg.predict_proba(X_test_no_outliers)[:, 1]

In [ ]:
_ = plot_roc(y_test_no_outliers, y_pred_proba_balanced)

In [ ]:
_ = predictions_hist(y_pred_proba_balanced, y_test_no_outliers)

In [ ]:
# invoque a função 'confusion_matrix_report' com um threshold que 
# otimize a precisão e o recall
<code>

### Lidando com classes desbalanceadas (parâmetro `class_weight`)

In [ ]:
# crie um objeto da classe LogisticRegression setando o parâmetro
# 'class_weight' com 'balanced'
<code>

In [ ]:
logreg.fit(X_train_no_outliers, y_train_no_outliers)
y_pred_proba_class_weight = logreg.predict_proba(X_test_no_outliers)[:, 1]

In [ ]:
_ = plot_roc(y_test_no_outliers, y_pred_proba_class_weight)

In [ ]:
_ = predictions_hist(y_pred_proba_class_weight, y_test_no_outliers)

## Salvando o modelo

Vamos salvar o modelo para conseguirmos carregá-lo em análises futuras pós-aula. Para detalhes, veja a documentação do scikit-learn: [Model Persistence](http://scikit-learn.org/stable/modules/model_persistence.html).

In [ ]:
_ = joblib.dump(logreg, '../models/logreg.pkl')

Vamos também salvar o dataset transformado, assim como foi utilizado pelo modelo final.

In [ ]:
(df_no_outliers.drop(columns='month')
               .to_csv('../data/bank_marketing_processed.csv', index=False))